Code to convert solar and wind mesh grid data to aggregate county data for modeling 

In [1]:
#imports
import os
import reverse_geocoder as rg
import numpy as np
import pandas as pd

cdat = pd.read_csv('Texas_Counties.csv', usecols=['County']) #all 254 counties in TX
counties = cdat['County'].to_list()

non_ercot_counties = ['El Paso','Hudspeth','Gaines','Terry','Yoakum','Cochran','Hockley','Lubbock','Bailey',
                      'Lamb','Hartley','Dallam','Moore','Sherman','Hansford','Hutchinson','Ochiltree','Lipscomb', 
                      'Hemphill','Bowie','Morris','Cass','Camp','Marion','Upshur','Gregg','Harrison','Panola',
                      'Shelby','San Augustine','Sabine','Trinity','Polk','Tyler','Jasper','Newton','San Jacinto',
                      'Hardin','Liberty','Orange','Jefferson']

remove_counties = [count + ' County' for count in non_ercot_counties]

for count in remove_counties:
    counties.remove(count)

print(len(counties))

213


In [ ]:
#pydrive quickstart google authorization - if you would like to use google drive for data storage
#consult pydrive quickstart instructions to set this up
from pydrive.auth import GoogleAuth

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # Creates local webserver and auto handles authentication.

#pydrive drive interaction
from pydrive.drive import GoogleDrive

drive = GoogleDrive(gauth)

In [ ]:
#solar reformatting

#get files from drive - put google drive folder id in parantheses
file_list = drive.ListFile({'q': "'1pqhBKFg6FGNOqTwyNs9dq5J2XChaWBUh' in parents and trashed=false"}).GetList()

#reformatting - want average capacity of all coordinates within county
countydic = dict.fromkeys(counties,np.zeros(17520)) #every half hour
countdic = dict.fromkeys(counties,0)
wcn = 0 #how many points in our mesh grid are not within an ERCOT county

for file in file_list:
    name = file['title'] #files are titled by coordinate
    lat = name.split('lat')[1]
    [lat,lon] = lat.split('lon')
    lon = lon.split('.csv')[0]
    lon = lon.split(' ')[0]
    coordinates = (float(lat),float(lon))
    county = rg.search(coordinates)[0]['admin2'] #reverse_geocoder used to site coordinates in county
    print(county)
    if county in counties:
        file = drive.CreateFile({'id': file['id']})
        file.GetContentFile('file.csv')
        pdat = pd.read_csv('file.csv', usecols=['PowerGen_kW'],dtype=np.float).to_numpy()
        pdat = [float(i) for i in pdat]
        countydic[county] = countydic[county] + pdat
        countdic[county] += 1
    else:
        wcn += 1

nameplate_cap = 22557.15 #nameplate capacity of used sample sites in kW
outdic = dict.fromkeys(counties,np.zeros(17520))
for county in counties:
    if countdic[county] != 0:
        #computing average solar capacity within each county
        outdic[county] = countydic[county] / (countdic[county]*nameplate_cap)

#df = pd.DataFrame(data=outdic).to_csv('Solar/Solar2011.csv')
print(len(file_list),wcn)

In [ ]:
#wind reformatting

#get files from drive - put google drive folder id in parantheses
file_list = drive.ListFile({'q': "'1H40oHl84pa8iaJ7xY4UpUlY78kvMlc2l' in parents and trashed=false"}).GetList()


#reformatting - max site per county
countydic = {}
sumdic = dict.fromkeys(counties,0) #keeping track of best site so far (in terms of average capacity factor)
#filedic = {} #keeping track of best location within each county - not necesary
wcn = 0 #how many points in our mesh grid are not within an ERCOT county

for file in file_list:
    name = file['title']
    lat = name.split('lat')[1]
    [lat,lon] = lat.split('lon')
    lon = lon.split('.csv')[0]
    coordinates = (float(lat),float(lon))
    county = rg.search(coordinates)[0]['admin2']
    if county in counties:
        file = drive.CreateFile({'id': file['id']})
        file.GetContentFile('file.csv')
        pdat = pd.read_csv('file.csv', skiprows=3, usecols=['Capacity Factor'],dtype=np.float, engine='python').to_numpy()
        dat = []
        ssum = 0
        for hour in range(17520):
            power = sum([float(i) for i in pdat[(6*hour):(6*(hour+1))]]) / 6 #5 min --> 30 min intervals
            dat.append(power)
            ssum += power
        #for getting max site
        if ssum > sumdic[county]: #is this site "better" than current best site in county
            countydic[county] = dat
            sumdic[county] = ssum
            #filedic[county] = [lat,lon]
    else:
        wcn += 1

#pd.DataFrame(data=countydic).to_csv('Wind/Wind2011.csv')
print(wcn)

In [2]:
#Coal reformatting - import coal data and format to be by the half hour
fuel = 'ERCOT/Fuel2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(fuel)

coalhours = np.empty(0)

for i in range(12):
    dat = pd.read_excel(xls, months[i])
    coal = dat.loc[dat['Fuel']=="Coal"]
    coal = coal[coal.columns[4::2]].to_numpy() + coal[coal.columns[5::2]].to_numpy() #every half hour
    coal = np.reshape(coal,np.prod(coal.shape))
    coalhours = np.concatenate((coalhours,coal))
coalhours = np.nan_to_num(coalhours)

pd.DataFrame(data={'Coal': coalhours}).to_csv('ERCOT/Coal2019.csv')

In [ ]:
#REGIONS (for doing regional analysis)
from uszipcode import SearchEngine
search = SearchEngine(simple_zipcode=True)

pdat = pd.read_excel('ERCOT/Zipcode_Data.xlsx', sheet_name='ZipToZone', usecols=['Svc. Address ZIP Code','Weather Zone Code'])

#removing zipcodes outside of ERCOT
update = pd.read_excel('ERCOT/Zipcode_Data_Update.xlsx', sheet_name='D1', skiprows=3, usecols=['Zipcode']).to_numpy().flatten()
dropidx = pdat.index[pdat['Svc. Address ZIP Code'].isin(update)].to_list()

pdat = pdat.drop(dropidx)

regions = np.array(['NORTH', 'NCENT', 'EAST', 'COAST', 'SOUTH', 'SCENT', 'WEST', 'FWEST']) #easier if numpy array
regdict = {'NORTH': [], 'NCENT': [], 'EAST': [], 'COAST': [], 'SOUTH': [], 'SCENT': [], 'WEST': [], 'FWEST': []}

for i,row in pdat.iterrows():
    zipcode = str(row['Svc. Address ZIP Code'])
    region = str(row['Weather Zone Code'])
    county = search.by_zipcode(zipcode).to_dict()['county']
    regdict[region].append(county)
    
#some counties in multiple regions - could put multiple region counties in primary region or just leave in multiple regions
#every county pretty dominantly in one county except austin (even split)
simpregdict = {}
multregdict = {}
for county in counties:
    counter = []
    for reg in regions:
        counter.append(regdict[reg].count(county))
    simpregion = regions[np.argmax(counter)]
    multregions = list(regions[[i for i, val in enumerate(counter) if val != 0]])
    simpregdict[county] = [simpregion] #for simple can do county --> region cause only 1 region per county
    multregdict[county] = multregions

simpregdict['Austin County'] = 'SCENT' #austin between two zones put in SCENT b/c map looks like that
pd.DataFrame(data=simpregdict).to_csv('CountyToRegion.csv')

In [5]:
#import coal plant data and format to be regional
pdat = pd.read_csv('EPA_Coal_2019.csv',usecols=['Facility_Name','Gross_Load_MW'])

oklaunion = pdat.loc[pdat['Facility_Name']=="Oklaunion Power Station"]['Gross_Load_MW'].to_numpy()
coletocreek = pdat.loc[pdat['Facility_Name']=="Coleto Creek"]['Gross_Load_MW'].to_numpy()
martinlake = pdat.loc[pdat['Facility_Name']=="Martin Lake"]['Gross_Load_MW'].to_numpy()
fayette = pdat.loc[pdat['Facility_Name']=="Sam Seymour"]['Gross_Load_MW'].to_numpy()
sanmiguel = pdat.loc[pdat['Facility_Name']=="San Miguel"]['Gross_Load_MW'].to_numpy()
sandycreek = pdat.loc[pdat['Facility_Name']=="Sandy Creek Energy Station"]['Gross_Load_MW'].to_numpy()
oakgrove = pdat.loc[pdat['Facility_Name']=="Oak Grove"]['Gross_Load_MW'].to_numpy()
jkspruce = pdat.loc[pdat['Facility_Name']=="J K Spruce"]['Gross_Load_MW'].to_numpy()
limestone = pdat.loc[pdat['Facility_Name']=="Limestone"]['Gross_Load_MW'].to_numpy()
waparish = pdat.loc[pdat['Facility_Name']=="W A Parish"]['Gross_Load_MW'].to_numpy()
twinoaks = pdat.loc[pdat['Facility_Name']=="Twin Oaks"]['Gross_Load_MW'].to_numpy()

#to convert gross output to net output, using top-level ratio of (net coal output in ERCOT / gross coal output in ERCOT)
coalgross = coletocreek + sanmiguel + jkspruce + fayette + limestone + sandycreek + oklaunion + martinlake + oakgrove + twinoaks + waparish
ratio = sum(coalhours) / sum(coalgross)
print(ratio)

#siting in a region based on county each plant is located within
southcoal = ratio*(coletocreek + sanmiguel)
scentcoal = ratio*(jkspruce + fayette)
ncentcoal = ratio*(limestone + sandycreek)
northcoal = ratio*(oklaunion)
eastcoal = ratio*(martinlake + oakgrove + twinoaks)
coastcoal = ratio*(waparish)
westcoal = np.zeros(8760)
fwestcoal = np.zeros(8760)

print(southcoal[0])
coalregs = {'NORTH': northcoal, 'NCENT': ncentcoal, 'EAST': eastcoal, 'COAST': coastcoal, 'SOUTH': southcoal, 'SCENT': scentcoal, 'WEST': westcoal, 'FWEST': fwestcoal}
pd.DataFrame(data=coalregs).to_csv('CoalRegional2019.csv')

0.925179186758031
517.1751653977393


In [ ]:
#import energy price data for each load region (different from weather regions)
#we are converting SPP2019 spreadsheet to useable halfhourly energy price data for each load region
#not using HB_HOUSTON (houston hub), COAST --> HB_SOUTH, NORTH --> HB_WEST
y = 17520
loadzones = ['HB_NORTH', 'HB_SOUTH', 'HB_WEST','HB_HUBAVG']
loadprices = dict.fromkeys(loadzones,np.empty(0))

epsheet = 'ERCOT/SPP2019.xlsx'
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
xls = pd.ExcelFile(epsheet)

for i in range(12):
    df = pd.read_excel(xls, months[i], usecols=['Settlement Point Name','Repeated Hour Flag','Settlement Point Price'])
    df = df.loc[df['Repeated Hour Flag']=='N']
    for zone in loadzones:
        eprice = df.loc[df['Settlement Point Name']==zone]['Settlement Point Price'].to_numpy()
        eprice = (eprice[::2] + eprice[1::2])/2 #15 min --> 30 min intervals
        loadprices[zone] = np.hstack((loadprices[zone],eprice))

#godam daylight savings time
for zone in loadzones:
    eprice = loadprices[zone]
    #issue with daylight savings time in data
    eprice = np.concatenate((eprice[0:3268],np.array([eprice[3267],eprice[3268]]),eprice[3268:17518]))
    loadprices[zone] = eprice

pd.DataFrame(data=loadprices).to_csv('ERCOT/EnergyPriceAgg2019.csv')